In [3]:
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import datetime
import shutil
from pathlib import Path
import trackpy
from iris.time import PartialDateTime
import cartopy.crs as ccrs
import xarray as xr
import netCDF4 as nc
import scipy
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure

#import packages that output memory usage:
#from sys import getsizeof

import tobac #tobac package cloned from https://github.com/tobac-project/tobac.git

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Import datasets
mask = xr.open_dataset('/data/users/hgilmour/tracking/code/tobac_sensitivity/Save/mask_2005_01.nc')
mask = mask.segmentation_mask
mask #segmentation mask from tracking on Jan 2005 with [240, 200] and [1975, 10]

precip = xr.open_dataset('/data/users/hgilmour/total_precip/precip_1h/precip_2005_01.nc')
precip = precip.stratiform_rainfall_flux #precip dataset for Jan 2005 (NEED TO LATER CONVERT FROM KG M-2 S-1 TO MM/HR (X3600))

tracks = pd.read_hdf('/data/users/hgilmour/tracking/code/tobac_sensitivity/Save/tracks_2005_01.h5', 'table')

tb = xr.open_dataset('/data/users/hgilmour/tb/2005/tb_2005_01.nc')
tb = tb.toa_outgoing_longwave_flux


In [5]:
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,forecast_reference_time,forecast_period,cell,time_cell
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,-64.032043,295.967957,295.967957,1,0 days 00:00:00
1,0,26,319.360864,876.726404,7359,240,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-27.105936,-49.532534,310.467466,310.467466,2,0 days 00:00:00
2,0,57,465.549812,1014.466367,5384,240,3,2005-01-01 00:30:00,2005-01-01 00:30:00,-21.185283,-43.954061,316.045939,316.045939,-1,NaT
3,0,73,763.319602,705.760312,47759,240,4,2005-01-01 00:30:00,2005-01-01 00:30:00,-9.125606,-56.456667,303.543333,303.543333,-1,NaT
4,0,86,577.470368,717.240701,2132,240,5,2005-01-01 00:30:00,2005-01-01 00:30:00,-16.652501,-55.991687,304.008313,304.008313,-1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9076,743,241,882.032521,698.342702,7910,240,9077,2005-01-31 23:30:00,2005-01-31 23:30:00,-4.317733,-56.757057,303.242943,303.242943,-1,NaT
9077,743,252,940.225439,833.293263,3228,240,9078,2005-01-31 23:30:00,2005-01-31 23:30:00,-1.960920,-51.291560,308.708440,308.708440,-1,NaT
9078,743,272,1046.778633,1275.959222,20583,240,9079,2005-01-31 23:30:00,2005-01-31 23:30:00,2.354484,-33.363590,326.636410,326.636410,-1,NaT
9079,743,320,1097.697734,916.296729,3373,240,9080,2005-01-31 23:30:00,2005-01-31 23:30:00,4.416709,-47.929939,312.070061,312.070061,-1,NaT


In [6]:
mask.shape == precip.shape # checking whether the mask and precip files have the same shape
# next steps won't work if not


True

In [7]:
# Copy tracks dataset into new tracks_precip dataset to append precip data to
tracks = tracks.copy()

In [21]:
# Add columns to the tracks_precip dataframe ready to append data to later
tracks['total_precip'] = 0
tracks['rain_flag'] = 0
tracks['convective_precip'] = 0
tracks['heavy_precip'] = 0
tracks['extreme_precip'] = 0
tracks['max_precip'] = 0
tracks['mean_precip'] = 0

# Add cold core filter columns
tracks['tb_min'] = 0
tracks['tb_mean'] = 0
tracks['cold_core_flag'] = 0
tracks['tb_210'] = 0
tracks['tb_200'] = 0
tracks['tb_190'] = 0

# # Add columns for colocated pixels of cold core <= 200K and precip > 1mm/hr
# tracks['colocated_pixels_lightrain'] = 0
# tracks['colocated_CC_proportion_lightrain'] = 0
# tracks['colocated_precip_proportion_lightrain'] = 0

# # Add columns for colocated pixels of cold core <= 200K and heavy_precip > 10mm/hr
# tracks['colocated_pixels_heavyrain'] = 0
# tracks['colocated_CC_proportion_heavyrain'] = 0
# tracks['colocated_precip_proportion_heavyrain'] = 0

# # Add columns for colocated pixels of cold core <= 200K and extreme precip > 50mm/hr
# tracks['colocated_pixels_extremerain'] = 0
# tracks['colocated_CC_proportion_extremerain'] = 0
# tracks['colocated_precip_proportion_extremerain'] = 0


In [22]:
# remove cell values with -1 from tracks dataset
tracks = tracks[tracks.cell >= 0]
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,...,tb_190,colocated_pixels_lightrain,colocated_CC_proportion_lightrain,colocated_precip_proportion_lightrain,colocated_pixels_heavyrain,colocated_CC_proportion_heavyrain,colocated_precip_proportion_heavyrain,colocated_pixels_extremerain,colocated_CC_proportion_extremerain,colocated_precip_proportion_extremerain
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,...,0,0,0,0,0,0,0,0,0,0
1,0,26,319.360864,876.726404,7359,240,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-27.105936,...,0,0,0,0,0,0,0,0,0,0
7,0,274,994.600931,652.721131,3251,240,8,2005-01-01 00:30:00,2005-01-01 00:30:00,0.241288,...,0,0,0,0,0,0,0,0,0,0
11,0,326,1136.057059,1330.881404,3448,240,12,2005-01-01 00:30:00,2005-01-01 00:30:00,5.970260,...,0,0,0,0,0,0,0,0,0,0
12,1,7,179.948634,523.925062,4138,240,13,2005-01-01 01:30:00,2005-01-01 01:30:00,-32.752130,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9062,742,351,1126.995562,1076.654673,15042,240,9063,2005-01-31 22:30:00,2005-01-31 22:30:00,5.603271,...,0,0,0,0,0,0,0,0,0,0
9064,743,28,506.670650,519.857317,40563,240,9065,2005-01-31 23:30:00,2005-01-31 23:30:00,-19.519889,...,0,0,0,0,0,0,0,0,0,0
9070,743,97,627.143818,585.244022,13066,240,9071,2005-01-31 23:30:00,2005-01-31 23:30:00,-14.640726,...,0,0,0,0,0,0,0,0,0,0
9074,743,208,801.606743,355.515817,3839,240,9075,2005-01-31 23:30:00,2005-01-31 23:30:00,-7.574979,...,0,0,0,0,0,0,0,0,0,0


In [23]:
tracks[tracks.cell == 8]

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,...,tb_190,colocated_pixels_lightrain,colocated_CC_proportion_lightrain,colocated_precip_proportion_lightrain,colocated_pixels_heavyrain,colocated_CC_proportion_heavyrain,colocated_precip_proportion_heavyrain,colocated_pixels_extremerain,colocated_CC_proportion_extremerain,colocated_precip_proportion_extremerain
7,0,274,994.600931,652.721131,3251,240,8,2005-01-01 00:30:00,2005-01-01 00:30:00,0.241288,...,0,0,0,0,0,0,0,0,0,0
23,1,246,997.302790,649.673525,3985,240,24,2005-01-01 01:30:00,2005-01-01 01:30:00,0.350713,...,0,0,0,0,0,0,0,0,0,0
40,2,207,1000.085394,647.171876,4462,240,41,2005-01-01 02:30:00,2005-01-01 02:30:00,0.463408,...,0,0,0,0,0,0,0,0,0,0
53,3,151,1003.611792,643.427105,4439,240,54,2005-01-01 03:30:00,2005-01-01 03:30:00,0.606227,...,0,0,0,0,0,0,0,0,0,0
68,4,153,1001.760375,637.916268,3516,240,69,2005-01-01 04:30:00,2005-01-01 04:30:00,0.531246,...,0,0,0,0,0,0,0,0,0,0
82,5,161,1002.426140,633.101328,1978,240,83,2005-01-01 05:30:00,2005-01-01 05:30:00,0.558210,...,0,0,0,0,0,0,0,0,0,0
97,6,178,999.329845,589.566796,2196,240,98,2005-01-01 06:30:00,2005-01-01 06:30:00,0.432808,...,0,0,0,0,0,0,0,0,0,0
110,7,166,1005.332832,573.307600,2083,240,111,2005-01-01 07:30:00,2005-01-01 07:30:00,0.675929,...,0,0,0,0,0,0,0,0,0,0


In [24]:
cells = np.unique(tracks.cell.values)
cells

array([   1,    2,    8,   12,   14,   27,   28,   45,   48,   50,   54,
         56,   57,   59,   60,   62,   64,   66,   70,   73,   89,   91,
         98,  104,  106,  116,  118,  122,  124,  127,  128,  130,  134,
        135,  137,  138,  151,  154,  162,  165,  178,  185,  195,  204,
        207,  212,  215,  222,  223,  224,  226,  234,  247,  271,  276,
        287,  293,  296,  306,  307,  309,  324,  326,  339,  345,  354,
        358,  364,  369,  371,  376,  396,  399,  406,  407,  408,  409,
        413,  414,  418,  429,  434,  437,  441,  442,  443,  445,  449,
        451,  453,  458,  459,  467,  492,  498,  500,  514,  516,  518,
        522,  524,  529,  539,  541,  544,  545,  548,  549,  551,  552,
        554,  558,  566,  567,  577,  584,  587,  588,  590,  596,  601,
        603,  605,  608,  611,  615,  619,  621,  622,  624,  630,  631,
        634,  636,  638,  646,  647,  652,  666,  670,  673,  677,  681,
        682,  689,  695,  698,  716,  721,  725,  7

In [49]:
## TESTING LOOP FOR A SINGLE UNIQUE CELL (E.G. NUMBER 12) ##

## rainfall thresholds to use within filtering loop ##
precip_threshold = 1 #mm/hr
heavy_precip_threshold = 10 # mm/hr
extreme_precip_threshold = 50 # mm/hr (based on Marengo, J. A., Ambrizzi, T., Alves, L. M., Barreto, N. J., Simões Reboita, M., & Ramos, A. M. (2020). Changing trends in rainfall extremes in the metropolitan area of São Paulo: causes and impacts. Frontiers in Climate, 2, 3.)
precip_area = 25 # threshold for the minimum number of grid points that must be precipitating for a track to remain (and not be dropped from the tracks dataset)

cold_threshold = 200

## other parameters that need to be defined before loop ##
s = generate_binary_structure(2,2) # need this in loop later on
removed = 0 # need this for loop later on


cell = 12
subset = tracks[tracks.cell == cell]
precipitation_flag = 0
cold_core_flag = 0

for feature in subset.feature.values: #find all the feature values for that unique cell / track (the feature value is a unique value for each feature in a frame /timestep)
    print("Feature:", feature)
    for frame in subset.frame[subset.feature == feature]: #find the frame / timestep that corresponds to the feature number
        print("Frame:", frame)
        if mask.shape == precip.shape:

            seg = mask[frame,:,:] #printing the segmentation mask which occurs in the same frame as the feature value
            #print(seg)
            prec = precip[frame,:,:] #printing the precip timesteps which occurs in the same frame as the feature value
            #print(prec)
            brightness_temp = tb[frame,:,:] # printing the tb timesteps which occurs in the same frame as the feature value

            featureid = subset.feature[subset.frame == frame].values[0] #find the feature number at each timestep / frame of the cells lifetime (it changes over time and doesn't stay constant)
            #print('featureid: {}'.format(featureid)) #we now know all the feature numbers that belong to a single cell over its lifetime
            
            
            labels, nr = ndimage.label(seg, structure = s) #this line uses ndimage package for image processing. It generates arrays of numbers and decides what are joined together and what aren't.
            # In other words, it does image segmentation tasks, such as finding connected components and labeling objects in an image.
            # (i.e. it generates the locations of all contiguous fields of the segmentation mask that belong to a specific cell at a specific timestep and gives it a label. The number of labels is also recorded (the number of segmented areas in the timestep))

            if featureid not in seg: #check that the feature id number at each timestep is within the segmentation mask, if not, it is ignored and we continue
                continue
            else:

                label = np.unique(labels[seg == featureid])[0] #put a label where the labels match for both the feature id and the segmentation mask
                seg_mask = seg.where(labels == label)

                #create coordinates from mask
                seg_mask.coords['mask'] = (('longitude', 'latitude'), seg_mask.data)

                #apply mask to precip dataset
                precip_values = prec.where(seg_mask.coords['mask'].values > 0) # creating a new dataset called 'precip_values' with only the precip values where the seg_mask pixel is labelled as greater than 0 (i.e. the MCS region)
                #print('precip values: {}'.format(precip_values)
                array = precip_values.values.flatten() * 3600 # precip values are converted to 1D numpy array and multiplied by 3600 to convert from kg m-2 s-1 to mm / hr
                values = array[~np.isnan(array)] #removes NaNs from the precip array for further calculations
                #print(values)

                total_precip = np.nansum(values[values > 0]) #working out the total precip associated with the mask. First, values of 0 are removed to only consider precipitating pixels. Then np.nansum is used to compute the sum of all precipitating values within the mask.
                #print('total precip: {}'.format(total_precip))
                subset['total_precip'][(subset.feature == featureid) & (subset.frame == frame)  & (subset.cell == cell)] = total_precip

                rain_features = values[values >= precip_threshold].shape[0] #number of pixels within the mask that meet the 1 mm/hr precip threshold
                #print('rain features: {}'.format(rain_features))
                subset['rain_flag'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = rain_features

                subset['convective_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = np.nansum(values[values >= precip_threshold]) #total rain from all pixel where the rainfall threshold of 1 mm/hr is met

                subset['heavy_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = np.nansum(values[values >= heavy_precip_threshold]) #total rain from all pixel where the heavy rainfall threshold of 10 mm/hr is met
                rain_features_heavy = values[values >= heavy_precip_threshold].shape[0] #number of pixels within the mask that meet the heavy rainfall threshold of 10 mm/hr

                subset['extreme_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = np.nansum(values[values >= extreme_precip_threshold]) #total rain from all pixel where the extreme rainfall threshold of 50 mm/hr is met
                rain_features_extreme = values[values >= extreme_precip_threshold].shape[0] #number of pixels within the mask that meet the extreme rainfall threshold of 50 mm/hr

                subset['max_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values.max()

                subset['mean_precip'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values.mean()


                # cold core filter using the same image processing mask ##
                values_tb = brightness_temp.where(seg_mask.coords['mask'].values > 0)
                array_tb = values_tb.values.flatten() # precip values are converted to 1D numpy array and multiplied by 3600 to convert from kg m-2 s-1 to mm / hr
                values_tb = array_tb[~np.isnan(array_tb)] #removes NaNs from the precip array for further calculations
                print(values_tb[values_tb <= 190].shape[0])
                #array = values_tb.to_dataframe
                #values_tb = array().toa_outgoing_longwave_flux #Tb values in 1D array format to use in section below:

                subset['tb_mean'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values_tb.mean()

                subset['tb_min'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = values_tb.min()

                subset['tb_210'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = (values_tb[values_tb <= 210]).shape[0]
                
                subset['tb_200'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = (values_tb[values_tb <= 200]).shape[0]             

                subset['tb_190'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = (values_tb[values_tb <= 190]).shape[0]              

                # Assigning cold core flag
                if values_tb.min() <= cold_threshold:
                    cold_core_flag += 1
                    subset['cold_core_flag'][(subset.feature == featureid) & (subset.frame == frame) & (subset.cell == cell)] = cold_core_flag

                if rain_features >= precip_area: # if the number of precipitating pixels exceeds the miniumum pixel number... 
                    precipitation_flag += rain_features # add rain pixels to the precipitation flag

if cold_core_flag == 0:
    subset = subset.drop(subset[subset.cell == cell].index)
    removed += 1 #print the number of tracks that have been removed from the original dataset

    #save subset to deleted tracks folder
    subset.to_hdf('Save/deleted_tracks/cold_core/tracks_2005_01_cell_{}'.format(cell), 'table')
    
if precipitation_flag == 0: #if the minumum precipitating pixel thresholds aren't met...
    #remove corresponding cell from the tracks dataframe
    subset = subset.drop(subset[subset.cell == cell].index)
    removed += 1 #print the number of tracks that have been removed from the original dataset

    #save subset to deleted tracks folder
    subset.to_hdf('Save/deleted_tracks/precip/tracks_2005_01_cell_{}'.format(cell), 'table')

else:    
    # save precip track files
    subset.to_hdf('Save/CC&PF/tracks_2005_01_cell_{}.h5'.format(cell), 'table')




        


Feature: 12
Frame: 0
0


KeyError: True

In [ ]:
subset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,...,tb_190,colocated_pixels_lightrain,colocated_CC_proportion_lightrain,colocated_precip_proportion_lightrain,colocated_pixels_heavyrain,colocated_CC_proportion_heavyrain,colocated_precip_proportion_heavyrain,colocated_pixels_extremerain,colocated_CC_proportion_extremerain,colocated_precip_proportion_extremerain
173,14,83,581.755098,527.987977,2052,240,174,2005-01-01 14:30:00,2005-01-01 14:30:00,-16.478970,...,1,0,0,0,0,0,0,0,0,0
184,15,112,617.767913,512.314843,4480,240,185,2005-01-01 15:30:00,2005-01-01 15:30:00,-15.020450,...,0,0,0,0,0,0,0,0,0,0
195,16,168,625.612071,500.962355,6725,240,196,2005-01-01 16:30:00,2005-01-01 16:30:00,-14.702761,...,0,0,0,0,0,0,0,0,0,0
208,17,195,627.969701,493.389162,10060,240,209,2005-01-01 17:30:00,2005-01-01 17:30:00,-14.607278,...,0,0,0,0,0,0,0,0,0,0
224,18,183,634.254754,498.907386,15442,240,225,2005-01-01 18:30:00,2005-01-01 18:30:00,-14.352733,...,0,0,0,0,0,0,0,0,0,0
245,19,167,636.161962,494.680687,19994,240,246,2005-01-01 19:30:00,2005-01-01 19:30:00,-14.275492,...,0,0,0,0,0,0,0,0,0,0
268,20,145,637.886552,492.881141,25094,240,269,2005-01-01 20:30:00,2005-01-01 20:30:00,-14.205645,...,0,0,0,0,0,0,0,0,0,0
290,21,120,635.293307,489.661063,27252,240,291,2005-01-01 21:30:00,2005-01-01 21:30:00,-14.310671,...,0,0,0,0,0,0,0,0,0,0
314,22,102,634.331350,475.479169,30398,240,315,2005-01-01 22:30:00,2005-01-01 22:30:00,-14.349631,...,0,0,0,0,0,0,0,0,0,0
338,23,84,650.941910,473.394967,37786,240,339,2005-01-01 23:30:00,2005-01-01 23:30:00,-13.676904,...,0,0,0,0,0,0,0,0,0,0
